In [50]:
import numpy as np
import random
import math
import pandas as pd

In [84]:
def euc_2d(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

def random_permutation(cities):
    perm = np.random.permutation(len(cities))
    return perm

def stochastic_two_opt(x):
    new_x = []
    x = list(x)
    start,end = sorted(random.sample(range(2,len(x)),2))
    new_x = x[:start]
    new_x += (list(reversed(x[start:end])))
    new_x += x[end:]
    
    return new_x

def augmented_cost(permutation, penalties, cities,lambda_):
    distance, augmented = 0, 0
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        d = euc_2d(cities[c1],cities[c2])
        distance += d
        augmented += d+(lambda_ * (penalties[c1][c2]))
    return [distance,augmented]

def cost(cand, penalties, cities, lambda_):
    cost, acost = augmented_cost(cand['vector'], penalties, cities, lambda_)
    cand['cost'], cand['aug_cost'] = cost, acost

def local_search(current, cities, penalties, max_no_improv, lambda_):
    cost(current, penalties, cities, lambda_)
    count = 0
    while count < max_no_improv:
        candidate = {'vector':stochastic_two_opt(current['vector'])}
        cost(candidate,penalties,cities,lambda_)
        if candidate['aug_cost'] < current['aug_cost']:
            count = 0 
        else:
            count += 1
        if candidate['aug_cost'] < current['aug_cost']:
            current = candidate           
    return current

def calculate_feature_utilities(penal, cities, permutation):
    utilities = [0] * len(permutation)
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        utilities[i] = euc_2d(cities[c1],cities[c2])/(1+penal[c1][c2])
    return utilities

def update_penalties(penalties, cities, permutation, utilities):
    max_ = max(utilities)
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        if utilities[i] == max_:
            penalties[c1][c2] += 1 
    return penalties

def search(max_iterations, cities, max_no_improv, lambda_):
    current = {'vector':random_permutation(cities)}
    best = current
    penalties = [ [0] * len(cities) for i in range(len(cities))]
    n = 0
    while n < max_iterations:
        current = local_search(current,cities,penalties,max_no_improv,lambda_)
        utilities = calculate_feature_utilities(penalties,cities,current['vector'])
        update_penalties(penalties,cities,current['vector'],utilities)
        if current['cost'] < best['cost']:
            best = current
            print(round(best['cost'],4))
        n +=1
    return best

In [85]:
# problem configuration
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]
# algorithm configuration
max_iterations = 15000
max_no_improv = 200
alpha = 0.3
local_search_optima = 12000.0
lambda_ = alpha * (local_search_optima/float(len(berlin52)))
# execute the algorithm
# best = search(max_iterations, berlin52, max_no_improv, lambda)

In [83]:
search(max_iterations, berlin52, max_no_improv, lambda_)

9956.3579
9923.8156
9916.3301
9672.4172
9619.3155
9538.9154
9492.7196
9386.2964
9377.6555
9342.0953
9268.9494
9268.6878
9264.9058
9260.3136
9253.3904
9040.4438
9031.1451
8958.3707
8935.4086
8932.2352
8927.6084


{'vector': [50,
  7,
  9,
  8,
  40,
  18,
  44,
  31,
  48,
  0,
  21,
  30,
  17,
  2,
  16,
  20,
  41,
  6,
  1,
  29,
  22,
  19,
  49,
  28,
  15,
  45,
  43,
  33,
  34,
  35,
  38,
  39,
  36,
  37,
  47,
  23,
  4,
  14,
  5,
  24,
  3,
  42,
  32,
  11,
  10,
  51,
  13,
  12,
  26,
  27,
  25,
  46],
 'cost': 8927.608378596877,
 'aug_cost': 16681.454532443036}